In [15]:
import pandas as pd
import os

In [20]:
# loop over all subjects
for i in range(1, 6):

    # 1 -> 001, ..., 10 -> 010
    sub = str(i).zfill(3)

    # loop over all runs 1 & 2
    for run in range(1,3):

        # load df
        df = pd.read_csv(f'{sub}/{sub}_run{run}.csv')

        # clean df
        df['trial_type'] = df['ImgPath'].apply(lambda x: 'house' if 'houses' in x else 'face' if 'faces' in x else None)
        df['onset'] = round(df['onset'], 2)
        df['duration'] = round(df['img_dur'], 1)
        df['end_time'] = round(df['end_time'], 2)
        df = df[['trial_type', 'onset', 'end_time', 'duration']] # keep only desired cols

        # block design df: 
        # create a new df with every 20 rows (each block)
        blocked_df = df.groupby(df.index // 20).apply(lambda x: pd.Series({
            'trial_type': x['trial_type'].unique()[0],
            'onset': round(x['onset'].iloc[0]),
            'duration': round(x['end_time'].iloc[-1] - x['onset'].iloc[0])
        })).reset_index(drop=True)


        # save cleaned df as csv
        save_sub_path = (f'cleaned_data/{sub}')
        if not os.path.exists(save_sub_path):
            os.makedirs(save_sub_path)
        
        blocked_df.to_csv(f'{save_sub_path}/sub-{sub}_task-localizer_run-{run}_events.csv')
